## TP2

Para la imagen suministrada “metal grid ”. Implementar un algoritmo
1. Calcule los gradientes en la imagen (dirección y módulo)
2. Muestre imágenes de ángulo y módulo
3. Marque con color las direcciones de gradientes más altos

